In [1]:
!pip install gradio duckduckgo-search newspaper3k transformers


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.3 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=e41c3eb9998c7c8a9647ab3eef31520554a10e2b79dda93a66e40ba767cefb66
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=30dbb848659c22c81e17ee11dffd96056550af1b97f6c4a2f6bf56e6e03f36e2
  Stored in d

In [2]:
from duckduckgo_search import DDGS

def search_news_links(company_name, max_results=5):
    query = f"{company_name} latest news"
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=max_results)]
        return [r["href"] for r in results if "href" in r]

# Test it (optional):
# print(search_news_links("Apple"))


In [5]:
!pip install newspaper3k lxml_html_clean


In [6]:
from newspaper import Article

def extract_news_content(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except:
        return ""


In [7]:
from transformers import pipeline

# Load an open-source summarizer
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize_text(text):
    if len(text) > 1000:
        text = text[:1000]  # Truncate if too long
    summary = summarizer(text, max_length=120, min_length=30, do_sample=False)
    return summary[0]['summary_text']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [8]:
import random

def is_greeting(text):
    greetings = ["hi", "hello", "hey", "how are you", "good morning", "good evening"]
    return any(greet in text.lower() for greet in greetings)

def handle_greeting():
    responses = ["Hey there!", "Hello! 👋", "Hi, how can I help you today?"]
    return random.choice(responses)

def get_news_summary(company_name, style="Formal"):
    links = search_news_links(company_name)
    if not links:
        return "Sorry, I couldn't find any news. Maybe check the spelling?"

    summaries = []
    for url in links:
        content = extract_news_content(url)
        if content:
            summary = summarize_text(content)
            if style == "Casual":
                summary = f"So guess what? Here's something new about {company_name}: {summary}"
            elif style == "Bullet Points":
                summary = "• " + summary.replace(". ", ".\n• ")
            summaries.append(f"{summary}\n[Read more]({url})")

    return "\n\n".join(summaries)


In [9]:
import gradio as gr

def chatbot_interface(user_input, style):
    if is_greeting(user_input):
        return handle_greeting()
    else:
        return get_news_summary(user_input, style)

style_options = ["Formal", "Casual", "Bullet Points"]

gr.Interface(fn=chatbot_interface,
             inputs=[gr.Textbox(label="Ask about any company..."),
                     gr.Radio(style_options, label="Choose your style")],
             outputs="text",
             title="📡 Smart Company News Chatbot 🤖",
             description="Get the latest news about any company with AI summaries!"
             ).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4f290665fc308d9953.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
